In [1]:
%matplotlib inline 

import pandas as pd
import utilities.misc as um
import utilities.constants as uc
import utilities.display as ud
import numpy as np
import matplotlib.pyplot as plt
import datetime


from twitter_api.twitter_api import get_all_tweets_from_friends

import re

import feather

from nltk_wrapper.nltk_wrapper import get_sentiment, clean_tokenize
    

In [2]:
# we can't save the dump with list
path="D:\\data\\twitter\\"

In [3]:
start=pd.datetime(2019,12,1)
end=um.today_date()

all_tweets=get_all_tweets_from_friends(start,end)

finished friend number 1/45, total tweet collected is 698
finished friend number 2/45, total tweet collected is 371
finished friend number 3/45, total tweet collected is 576
finished friend number 4/45, total tweet collected is 468
finished friend number 5/45, total tweet collected is 907
finished friend number 6/45, total tweet collected is 2582
finished friend number 7/45, total tweet collected is 738
finished friend number 8/45, total tweet collected is 97
finished friend number 9/45, total tweet collected is 2127
finished friend number 10/45, total tweet collected is 4
finished friend number 11/45, total tweet collected is 348
finished friend number 12/45, total tweet collected is 343
finished friend number 13/45, total tweet collected is 1099
finished friend number 14/45, total tweet collected is 747
finished friend number 15/45, total tweet collected is 1092
finished friend number 16/45, total tweet collected is 12
finished friend number 17/45, total tweet collected is 11
finishe

In [4]:
study=all_tweets[all_tweets['lang']=='en'].copy()
study['sentiment']=study['text'].map(lambda x: get_sentiment(x))

In [5]:
# Get data by hashtag
study_hash=study[study['hashtags'].map(lambda x: len(x))>0]
collector=[]
for i,row in enumerate(study_hash.index):
    info_i=study_hash.iloc[i]
    hash_i=info_i['hashtags']
    date_i=info_i['date']
    retweets_i=info_i['retweets']
    likes_i=info_i['likes']
    
    res_i=pd.DataFrame(index=hash_i,columns=['date','retweets','likes'],data=[[date_i,retweets_i,likes_i]])
    collector.append(res_i)
hash_all=pd.concat(collector)
hash_all.index.name='hashtag'

In [6]:
# get the word cloud
collector=[]
for i, idx in enumerate(study.index):
    info_i=study.iloc[i]
    tweet_i=clean_tokenize(info_i['text'])
    res_i=pd.Series(index=np.arange(0,len(tweet_i)),data=tweet_i).rename('word').to_frame()
    res_i['date']=info_i['date']
    res_i['tweet_id']=info_i['tweet_id']
    res_i['likes']=info_i['likes']
    res_i['retweets']=info_i['retweets']
    collector.append(res_i)
words=pd.concat(collector,axis=0)

In [7]:
# dump some data
feather.write_dataframe(hash_all.reset_index(),path+'hash.feather')
feather.write_dataframe(.drop(['hashtags','mentions_id','mentions_name'],axis=1),path+'text.feather')
feather.write_dataframe(words,path+'words.feather')